In [4]:
from bayes_opt import BayesianOptimization
import pandas as pd
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import matplotlib.pyplot as plt
from sklearn.model_selection import ShuffleSplit, cross_val_score
import numpy as np
from sklearn.model_selection import GroupShuffleSplit, GroupKFold
from skopt.space import Real, Categorical, Integer
from skopt import BayesSearchCV
from xgboost import XGBRegressor
import pickle

In [5]:
def evaluate_performance(best_model, X, y_true):
    y_pred = best_model.predict(X)

    mae = mean_absolute_error(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)

    performance = {
        'MAE': mae,
        'MSE': mse,
        'R2': r2,
    }
    return performance

In [6]:
# Load data
data = pd.read_csv("data02.csv")
X = data.iloc[:, 2:]
y = data['value']
group = data['group']

# Initialize lists to store performance metrics
train_performance = []
val_performance = []
test_performance = []

all_test_preds = []

gkf = GroupKFold(n_splits=10)

val_test_diff_list = []
param_list = []

round_id = 1

# Loop through each fold
for train_index, test_index in gkf.split(X, y, group):
    print(f"\nNew round {round_id}")
    round_id += 1
    
    remain_data = data.iloc[train_index]
    test_data = data.iloc[test_index]

    X_remain=remain_data.iloc[:, 2:]
    y_remain=remain_data['value']
    group_remain=remain_data['group']

    X_test=test_data.iloc[:, 2:]
    y_test=test_data['value']
    group_test=test_data['group']

    # XGBoost 参数空间
    p_grid = {
        'booster': Categorical(['gbtree']),
        'n_estimators': Integer(100, 200),
        'max_depth': Integer(4, 8),
        'gamma': Integer(0, 5),
        'learning_rate': Real(0.05, 0.2, prior='log-uniform'),
        'subsample': Real(0.5, 1.0),
        'min_child_weight': Real(1.0, 3.0),
        'max_delta_step': Integer(1, 5),
        'reg_alpha': Real(0.01, 0.1, prior='log-uniform'),
        'reg_lambda': Real(0.01, 0.1, prior='log-uniform')
    }

    # configure the cross-validation procedure - inner loop (validation set/HP optimization)
    cv_inner = GroupKFold(n_splits=10)  

    model = XGBRegressor(random_state=42, n_jobs=-1, tree_method='hist', verbosity=0 )

    # define search space
    search = BayesSearchCV(model, p_grid, n_iter=100, verbose=0, scoring='r2', cv=cv_inner,  n_jobs= 1, refit=True) # should be 100
    
    # execute search
    result = search.fit(X_remain, y_remain, groups=group_remain)
        
    # get the best performing model fit on the whole training set
    # 十次交叉验证找出最优参数
    best_model = result.best_estimator_           

    # Fit the best model on the entire training set
    # 在整个训练集上拟合最优模型best_model_remain，得到训练集和测试集的性能
    best_model_remain=best_model.fit(X_remain, y_remain)
    performance_train = evaluate_performance(best_model_remain, X_remain, y_remain)
    performance_test = evaluate_performance(best_model_remain, X_test, y_test)

    # 添加 test 预测结果
    y_test_pred = best_model_remain.predict(X_test)
    test_pred_df = pd.DataFrame({
        'true_value': y_test.values,
        'predicted_value': y_test_pred,
        'group': test_data['group'].values,  # 可选：保留 group 信息
        'time': test_data['Time'].values  # 可选：保留时间信息
    })
    all_test_preds.append(test_pred_df)
        
    allfold_performance_val = []
    
    # 记录验证集结果，在内层X_remain交叉验证中计算验证集性能
    for train_idx, valid_idx in cv_inner.split(X_remain, y_remain, groups=group_remain):
        X_train = X_remain.iloc[train_idx]
        y_train = y_remain.iloc[train_idx]
        X_val = X_remain.iloc[valid_idx]
        y_val = y_remain.iloc[valid_idx]

        model_train = best_model.fit(X_train, y_train)
        
        fold_performance_val = evaluate_performance(model_train, X_val, y_val)

        allfold_performance_val.append(fold_performance_val)

        performance_val = pd.DataFrame(allfold_performance_val).mean()

    # 存储性能
    train_performance.append(performance_train)
    val_performance.append(performance_val)
    test_performance.append(performance_test)
    
    print("performance_train=")
    print(performance_train)
    print("performance_val=")
    print(performance_val)
    print("performance_test=")
    print(performance_test)

    val_r2 = performance_val['R2']
    test_r2 = performance_test['R2']
    val_test_diff = abs(val_r2 - test_r2)
    val_test_diff_list.append(val_test_diff)
    param_list.append(result.best_params_)


# 汇总10折预测结果
final_test_result = pd.concat(all_test_preds, ignore_index=True)
final_test_result.to_csv('test_results_XGB.csv', index=False)

# Calculate average performance metrics
avg_train_performance = pd.DataFrame(train_performance).mean()
avg_val_performance = pd.DataFrame(val_performance).mean()
avg_test_performance = pd.DataFrame(test_performance).mean()
std_train_performance = pd.DataFrame(train_performance).std()
std_val_performance = pd.DataFrame(val_performance).std()
std_test_performance = pd.DataFrame(test_performance).std()

# Print av erage performance metrics
print("\n" +"Average Train Performance:" + str(avg_train_performance) + "+/-" + str(std_train_performance))
print("\n" +"Average Validation Performance:", avg_val_performance, "+/-", std_val_performance)
print("\n" +"Average Test Performance:", avg_test_performance, "+/-", std_test_performance)


New round 1
performance_train=
{'MAE': 0.22821062281586083, 'MSE': 0.11468405424894636, 'R2': 0.8608302290689775}
performance_val=
MAE    0.309504
MSE    0.235155
R2     0.708955
dtype: float64
performance_test=
{'MAE': 0.3321175544866298, 'MSE': 0.2739689884446958, 'R2': 0.5596879226743667}

New round 2
performance_train=
{'MAE': 0.1948872358916255, 'MSE': 0.08453990358485072, 'R2': 0.8927330879336968}
performance_val=
MAE    0.315633
MSE    0.236775
R2     0.687687
dtype: float64
performance_test=
{'MAE': 0.28978997234686044, 'MSE': 0.19125412842516337, 'R2': 0.7991585102862795}

New round 3
performance_train=
{'MAE': 0.22155717312260106, 'MSE': 0.11073740980139324, 'R2': 0.8597888562341804}
performance_val=
MAE    0.306263
MSE    0.231461
R2     0.705501
dtype: float64
performance_test=
{'MAE': 0.3098855119009934, 'MSE': 0.21137819666150645, 'R2': 0.7739859559269274}

New round 4
performance_train=
{'MAE': 0.2226085249502904, 'MSE': 0.1128006629281559, 'R2': 0.8628480540317351}
per

In [7]:
# 创建性能汇总 DataFrame（包含最优参数）
performance_by_fold = pd.DataFrame({
    'Fold': list(range(1, len(train_performance) + 1)),
    'Train_R2': [p['R2'] for p in train_performance],
    'Val_R2': [p['R2'] for p in val_performance],
    'Test_R2': [p['R2'] for p in test_performance],
    'Train_MAE': [p['MAE'] for p in train_performance],
    'Val_MAE': [p['MAE'] for p in val_performance],
    'Test_MAE': [p['MAE'] for p in test_performance],
    'Train_MSE': [p['MSE'] for p in train_performance],
    'Val_MSE': [p['MSE'] for p in val_performance],
    'Test_MSE': [p['MSE'] for p in test_performance],
    'Val_Test_R2_Diff': val_test_diff_list,
    'Best_Params': param_list
})

# 保存为 CSV
performance_by_fold.to_csv("XGB_results.csv", index=False)

In [8]:
# 找到 val-test 差值最小的一折
best_param_idx = int(np.argmin(val_test_diff_list))
best_params = param_list[best_param_idx]

# 全部数据重新训练最终模型
final_model = XGBRegressor(**best_params)
final_model.fit(X, y)

# 保存模型为 pickle
import pickle
with open("XGB_model.pkl", "wb") as f:
    pickle.dump(final_model, f)

print(f"\n✅ 最优模型已保存：第 {best_param_idx + 1} 折参数（val-test最接近）用于全数据训练")



✅ 最优模型已保存：第 6 折参数（val-test最接近）用于全数据训练
